In [1]:
import requests
import re
import json
import math
import time
import random

def rsa_no_padding(src, modulus, exponent):
    m = int(modulus, 16)
    e = int(exponent, 16)
    t = bytes(src, 'ascii')
    # 字符串转换为bytes
    input_nr = int.from_bytes(t, byteorder='big')
    # 将字节转化成int型数字，如果没有标明进制，看做ascii码值
    crypt_nr = pow(input_nr, e, m)
    # 计算x的y次方，如果z在存在，则再对结果进行取模，其结果等效于pow(x,y) %z
    length = math.ceil(m.bit_length() / 8)
    # 取模数的比特长度(二进制长度)，除以8将比特转为字节
    crypt_data = crypt_nr.to_bytes(length, byteorder='big')
    # 将密文转换为bytes存储(8字节)，返回hex(16字节)
    return crypt_data.hex()


In [2]:
def updatescore():
    session = requests.session()

    # 打开网站
    res = session.get('https://zjuam.zju.edu.cn/cas/login?service=http://zdbk.zju.edu.cn/jwglxt/xtgl/login_ssologin.html')
    # 获取execution的值以用于登录
    execution = re.findall(r'<input type="hidden" name="execution" value="(.*?)" />', res.text)[0]
    # 获取RSA公钥
    res = session.get('https://zjuam.zju.edu.cn/cas/v2/getPubKey')
    modulus = res.json()['modulus']
    exponent = res.json()['exponent']

    with open('database.json', 'r') as f:
        userdata = json.load(f)
    username = userdata['username']
    password = userdata['password']
    url = userdata.get('url', 'https://oapi.dingtalk.com/robot/send?access_token=')

    rsapwd = rsa_no_padding(password, modulus, exponent)

    data = {
        'username': username,
        'password': rsapwd,
        'execution': execution,
        '_eventId': 'submit'
    }
    # 登录
    res = session.post('https://zjuam.zju.edu.cn/cas/login?service=http://zdbk.zju.edu.cn/jwglxt/xtgl/login_ssologin.html', data)
    
    gnmkdm = 'N5083'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Linux; Android 10; Redmi K30 Pro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Mobile Safari/537.36',
    }

    res = session.post(url=f'http://zdbk.zju.edu.cn/jwglxt/cxdy/xscjcx_cxXscjIndex.html?doType=query&gnmkdm={gnmkdm}&su={username}', data={
        'xn': None,
        'xq': None,
        'zscjl': None,
        'zscjr': None,
        '_search': 'false',
        'nd': str(int(time.time() * 1000)),
        'queryModel.showCount': 5000,
        'queryModel.currentPage': 1,
        'queryModel.sortName': 'xkkh',
        'queryModel.sortOrder': 'asc',
        'time': 0,
    }, headers=headers)

    new_score = res.json()['items']
    
    try:
        with open("dingscore.json", 'r') as load_f:
            userscore = json.load(load_f)
    except json.decoder.JSONDecodeError:
        userscore = {}
    except FileNotFoundError:
        userscore = {}

    totcredits = 0
    totgp = 0
    for lesson in userscore:
        if userscore[lesson]['score'] in ['合格', '不合格', '弃修']:
            continue
        totgp += float(userscore[lesson]['gp']) * float(userscore[lesson]['credit'])
        totcredits += float(userscore[lesson]['credit'])
    try:
        gpa = totgp / totcredits
    except:
        gpa = 0
    
    #对比以更新
    for lesson in new_score:
        id = lesson['xkkh']
        name = lesson['kcmc']
        score = lesson['cj']
        credit = lesson['xf']
        gp = lesson['jd']
        if id == '选课课号':
            continue
        if userscore.get(id) != None:
            continue
        
        #新的成绩更新
        userscore[id] = {
            'name': name,
            'score': score,
            'credit': credit,
            'gp': gp
        }
        newtotcredits = 0
        newtotgp = 0
        for lesson in userscore:
            if userscore[lesson]['score'] in ['合格', '不合格', '弃修']:
                continue
            newtotgp += float(userscore[lesson]['gp']) * float(userscore[lesson]['credit'])
            newtotcredits += float(userscore[lesson]['credit'])
        try:
            newgpa = newtotgp / newtotcredits
        except:
            newgpa = 0
        
        #钉钉推送消息
        try:
            requests.post(url=url, json={
                "msgtype": "markdown",
                "markdown" : {
                    "title": "考试成绩通知",
                    "text": """
### 考试成绩通知\n
 - **选课课号**\t%s\n
 - **课程名称**\t%s\n
 - **成绩**\t%s\n
 - **学分**\t%s\n
 - **绩点**\t%s\n
 - **成绩变化**\t%.2f(%+.2f) / %.1f(%+.1f)""" % (id, name, score, credit, gp, newgpa, newgpa - gpa, newtotcredits, newtotcredits - totcredits)
                }
            })
        except requests.exceptions.MissingSchema:
            print('The DingTalk Webhook URL is invalid. Please use -d [DingWebhook] to reset it first.')
        
        print('考试成绩通知\n选课课号\t%s\n课程名称\t%s\n成绩\t%s\n学分\t%s\n绩点\t%s\n成绩变化\t%.2f(%+.2f) / %.1f(%+.1f)' % (id, name, score, credit, gp, newgpa, newgpa - gpa, newtotcredits, newtotcredits - totcredits))
        totcredits = newtotcredits
        totgp = newtotgp
        gpa = newgpa

    #保存新的数据
    with open("dingscore.json", 'w', encoding="utf-8") as load_f:
        load_f.write(json.dumps(userscore, indent=4, ensure_ascii=False))

def scorenotification():
    while True:
        try:
            updatescore()
        except Exception as e:
            print(time.ctime() + " " + str(e))
        finally:
            time.sleep(random.randint(60, 300))

In [32]:
session = requests.session()

# 打开网站
res = session.get('https://zjuam.zju.edu.cn/cas/login?service=https://zdbk.zju.edu.cn/jwglxt/xtgl/login_ssologin.html')
# 获取execution的值以用于登录
execution = re.findall(r'<input type="hidden" name="execution" value="(.*?)" />', res.text)[0]
# 获取RSA公钥
res = session.get('https://zjuam.zju.edu.cn/cas/v2/getPubKey')
modulus = res.json()['modulus']
exponent = res.json()['exponent']

with open('database.json', 'r') as f:
    userdata = json.load(f)
username = userdata['username']
password = userdata['password']
url = userdata.get('url', 'https://oapi.dingtalk.com/robot/send?access_token=')

rsapwd = rsa_no_padding(password, modulus, exponent)

data = {
    'username': username,
    'password': rsapwd,
    'execution': execution,
    '_eventId': 'submit'
}
# 登录
res = session.post('https://zjuam.zju.edu.cn/cas/login?service=https://zdbk.zju.edu.cn/jwglxt/xtgl/login_ssologin.html', data)
iPlanetDirectoryPro = res.cookies.get('iPlanetDirectoryPro')
session.cookies.set('iPlanetDirectoryPro', iPlanetDirectoryPro)

res = session.get("https://zjuam.zju.edu.cn/cas/login?service=http%3A%2F%2Fzdbk.zju.edu.cn%2Fjwglxt%2Fxtgl%2Flogin_ssologin.html")
print(res.headers)

{'Server': 'nginx/1.20.1', 'Date': 'Mon, 23 Jun 2025 06:14:17 GMT', 'Content-Type': 'text/html;charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Cache-Control': 'no-store', 'Set-Cookie': '_csrf=S8mwplVi9KWoF2WQ0TlCeJesn7hURNRVBqWT9MoVtVQ%3D; Domain=zju.edu.cn; Path=/', 'vary': 'accept-encoding', 'Content-Encoding': 'gzip'}


In [9]:
len(new_score)

10

In [ ]:
session = requests.session()

# 打开网站
res = session.get('https://zjuam.zju.edu.cn/cas/login?service=https://zdbk.zju.edu.cn/jwglxt/xtgl/login_ssologin.html')

# 获取execution的值以用于登录
execution = re.findall(r'<input type="hidden" name="execution" value="(.*?)" />', res.text)[0]
# 获取RSA公钥
res = session.get('https://zjuam.zju.edu.cn/cas/v2/getPubKey')
modulus = res.json()['modulus']
exponent = res.json()['exponent']



rsapwd = rsa_no_padding(password, modulus, exponent)

data = {
    'username': username,
    'password': rsapwd,
    'execution': execution,
    '_eventId': 'submit'
}

# 登录
res = session.post('https://zjuam.zju.edu.cn/cas/login', data)
print(session.cookies)
print(res.headers)

# res = session.get("https://zjuam.zju.edu.cn/cas/login?service=http%3A%2F%2Fzdbk.zju.edu.cn%2Fjwglxt%2Fxtgl%2Flogin_ssologin.html")
# print(res.headers)


<RequestsCookieJar[<Cookie _csrf=S8mwplVi9KWoF2WQ0TlCeDwkpo8qvh%2F7MpX5E7MQHTU%3D for .zju.edu.cn/>, <Cookie _pc0=1Lb7FNzCWpX92qZfMX0YAMwCCR3mL8JNxho9HFNjn%2BvGKguMcj9XdCKZAZEKhPkv for .zju.edu.cn/>, <Cookie _pf0=tg6EgoBucIOJ%2FcecITvG4o0A8ixGykxlTW1foA4wyTM%3D for .zju.edu.cn/>, <Cookie _pv0=FPrNRrrJ0uV%2FjaaZsDSvVd136v3exCvv2YNW%2BbiM52h%2BcEtqEDxIg6RAgaJWLzUXNe3NshzLN9NuFAvfpr11B94LbCy2CGu%2FeooxeU%2B9EQt6KTzFh8nquxMn9bbo34jrpqk2V%2FdKrSA6S%2FijEXgY%2F1sFz41vp9nFpz75qqBiDPegYVERrEZSjAY8YTwPhBa9DeU17MI3zRqK8lr7MvGkdL1%2BQ8%2BxiGJC7U1fBO7FzJc9AjYtaRRU6hMJZR1x7x%2BeB5a0ock3CfbLeryw8GpefAJ2D2tr2UOh4dy%2FkV23t8P8doGQQLh3hws99Pe81TAsK44M0D0EmEAG3rULyVMrAKiq%2BDw%2BL2%2Bb06HRLppdRELIgfSbygqxnMvgOPF5tHmHAjXMlB6R3pqePLvSfuZxN06gFsmuUlGBGKaoXpbroVg%3D for .zju.edu.cn/>, <Cookie iPlanetDirectoryPro=liV%2FfzvhDJALzEIWORFxsksfaoKy%2BJEDeV76cTVoo0X%2FSTXsqVf1Nk%2BqrqoIj3Yksjv9vI4hrOob7y84gBSbM8Jj1C2XYhYe39PtS2FTjsHp%2FXq2gq0HPlJ1hzj46ho9yX9YnDNW8ZcU4VfVjN%2BoJP7E7jBlB1UvzjVtbhFXo9LC49YpCFcUK%2FS7

In [44]:
for co in res.cookies:
    print(co.name,)

language


In [48]:
def get_sso_cookie(username: str, password: str):
    session = requests.Session()
    session.headers.update({
        "User-Agent": "Mozilla/5.0"
    })

    try:
        # Step 1: 获取登录页面，提取 execution 值
        resp = session.get('https://zjuam.zju.edu.cn/cas/login', timeout=8, allow_redirects=False)
        execution_match = re.search(r'name="execution" value="(.*?)"', resp.text)
        if not execution_match:
            raise "无法获取execution"
        execution = execution_match.group(1)

        # Step 2: 获取 RSA 公钥
        pubkey_resp = session.get('https://zjuam.zju.edu.cn/cas/v2/getPubKey', timeout=8)
        pubkey_json = pubkey_resp.json()
        modulus_str = pubkey_json.get("modulus")
        exponent_str = pubkey_json.get("exponent")
        if not modulus_str or not exponent_str:
            raise "无法获取RSA公钥"

        # Step 3: 执行 RSA 加密
        try:
            mod_int = int(modulus_str, 16)
            exp_int = int(exponent_str, 16)
            pwd_bytes = password.encode("utf-8")
            pwd_int = int(pwd_bytes.hex(), 16)
            pwd_enc_int = pow(pwd_int, exp_int, mod_int)
            pwd_enc = format(pwd_enc_int, 'x').zfill(128)
        except Exception:
            raise "密码不合法"

        # Step 4: 提交登录表单
        data = {
            "username": username,
            "password": pwd_enc,
            "execution": execution,
            "_eventId": "submit",
            "rememberMe": "true"
        }
        headers = {
            "Content-Type": "application/x-www-form-urlencoded; charset=utf-8"
        }
        login_resp = session.post(
            'https://zjuam.zju.edu.cn/cas/login',
            data=data,
            headers=headers,
            timeout=8,
            allow_redirects=False
        )

        # Step 5: 检查是否登录成功（是否有 iPlanetDirectoryPro Cookie）
        for cookie in session.cookies:
            if cookie.name == "iPlanetDirectoryPro":
                return cookie

        raise "学号或密码错误"

    except requests.exceptions.Timeout:
        raise "请求超时"
    except requests.exceptions.RequestException:
        raise "网络错误"

In [49]:
with open('database.json', 'r') as f:
    userdata = json.load(f)
username = userdata['username']
password = userdata['password']
url = userdata.get('url', 'https://oapi.dingtalk.com/robot/send?access_token=')

iPlanetDirectoryPro = get_sso_cookie(username, password)


In [50]:
def login(iPlanetDirectoryPro) -> bool:
    if iPlanetDirectoryPro is None:
        raise "iPlanetDirectoryPro无效"

    session = requests.Session()
    session.headers.update({"User-Agent": "Mozilla/5.0"})

    # 设置 iPlanetDirectoryPro cookie
    session.cookies.set(
        name=iPlanetDirectoryPro.name,
        value=iPlanetDirectoryPro.value,
        domain="zjuam.zju.edu.cn"
    )

    try:
        # Step 1: 访问 login 接口获取 st ticket
        url = "https://zjuam.zju.edu.cn/cas/login?service=http%3A%2F%2Fzdbk.zju.edu.cn%2Fjwglxt%2Fxtgl%2Flogin_ssologin.html"
        resp = session.get(url, timeout=8, allow_redirects=False)

        # 获取跳转地址
        st_location = resp.headers.get("Location")
        if not st_location:
            raise "iPlanetDirectoryPro无效"

        if st_location.startswith("http://"):
            st_location = st_location.replace("http://", "https://")

        # Step 2: 跟随跳转
        resp2 = session.get(st_location, timeout=8, allow_redirects=False)

        # 检查 cookies
        global jsessionid, route
        jsessionid = None
        route = None
        for cookie in session.cookies:
            if cookie.name == "JSESSIONID" and cookie.path == "/jwglxt":
                jsessionid = cookie.value
            elif cookie.name == "route":
                route = cookie.value

        if not jsessionid:
            raise "无法获取JSESSIONID"
        if not route:
            raise "无法获取route"

        return True

    except requests.exceptions.Timeout:
        raise "请求超时"
    except requests.exceptions.RequestException:
        raise "网络错误"

In [51]:
login(iPlanetDirectoryPro)

True

In [ ]:
def query_grades():
    if jsessionid is None or route is None:
        raise "未登录"

    session = requests.Session()
    session.headers.update({
        "User-Agent": "Mozilla/5.0"
    })

    # 设置 cookies
    session.cookies.set("JSESSIONID", jsessionid, domain="zdbk.zju.edu.cn", path="/jwglxt")
    session.cookies.set("route", route, domain="zdbk.zju.edu.cn")

    try:
        url = (
            "https://zdbk.zju.edu.cn/jwglxt/cxdy/xscjcx_cxXscjIndex.html"
            "?doType=query&queryModel.showCount=5000"
        )
        response = session.post(url, timeout=8, allow_redirects=False)

        return response

    except requests.exceptions.Timeout:
        raise "请求超时"
    except requests.exceptions.RequestException:
        raise "网络错误"

In [54]:
res = query_grades()

In [56]:
res.json()['items']

[{'cj': '合格',
  'completeAnswer': True,
  'jd': '3.0',
  'jgpxzd': '1',
  'kcmc': '英语水平测试',
  'listnav': 'false',
  'localeKey': 'zh_CN',
  'pageable': True,
  'queryModel': {'currentPage': 1,
   'currentResult': 0,
   'entityOrField': False,
   'limit': 15,
   'offset': 0,
   'pageNo': 0,
   'pageSize': 15,
   'showCount': 10,
   'sorts': [],
   'totalCount': 0,
   'totalPage': 0,
   'totalResult': 0},
  'rangeable': True,
  'row_id': '1',
  'totalResult': '64',
  'userModel': {'monitor': False,
   'roleCount': 0,
   'roleKeys': '',
   'roleValues': '',
   'status': 0,
   'usable': False},
  'xf': '1.0',
  'xkkh': '(2023-2024-2)-051F0600-000000-1'},
 {'cj': '合格',
  'completeAnswer': True,
  'jd': '3.0',
  'jgpxzd': '1',
  'kcmc': '艺术实训与展演——灵韵合唱Ⅰ',
  'listnav': 'false',
  'localeKey': 'zh_CN',
  'pageable': True,
  'queryModel': {'currentPage': 1,
   'currentResult': 0,
   'entityOrField': False,
   'limit': 15,
   'offset': 0,
   'pageNo': 0,
   'pageSize': 15,
   'showCount': 10,
   

In [52]:
jsessionid, route

('674C040925B111FE4CE4988CF8CA262B', '1a53339a9972202950f42a60e12340ac')

<RequestsCookieJar[<Cookie _csrf=S8mwplVi9KWoF2WQ0TlCeJMD5V7bevKQ%2Bsgw0%2BTB3iI%3D for .zju.edu.cn/>, <Cookie _pc0=1Lb7FNzCWpX92qZfMX0YAKb5kI837r%2FFiBo%2FCbJ7CmBfsINB4rkctiHWihfsA8vS for .zju.edu.cn/>, <Cookie _pf0=tg6EgoBucIOJ%2FcecITvG4gW9D0bmC%2BdLZpzrcvi6PSw%3D for .zju.edu.cn/>, <Cookie _pv0=hD%2FkX5EmVZI2qXqKBnRzfHdLXvGyp2GoGzSwdGtbs6hFZPx%2Ft5TxH0SGPIkMnj70ZSPrnLUsKSz0D5G6Pqox%2BSydH1sAGdWt97%2FGmUAzHKsYPn%2BW%2BHk%2FU5w6nRCiJTHLoDVJBIt4vCxHRQ4FVwRhA0Cr3%2FaM1pErKYo6rHhjq7Iu5PcTjUkGPJRtiiCAMXxdW8ghJFFxVJVyTyLmtUDBqTLTCQOf0yewSNWttdnQk8x2nd%2FaLjzg8h%2F%2BU0OhknFWVhLHOUK1J%2BiLibAphQj8XPYscdsOo35wUBrtN3ClVAVpuZjW0KpZhXAwhWZt%2FIMwjzR7Z6CHVqLbix3Q9Fz2rgsBcRb1ghB1U0bGK4EhczrGuRvCH8ugGb9pBkXkOwabGLli%2FJIRISX%2BnYjjLTFjSXEbp%2BEYlGr4LUMYMRWllM4%3D for .zju.edu.cn/>, <Cookie iPlanetDirectoryPro=iipmjEgdi8wxgCrLc5XDQleZSbwpSXqm%2FP23XV1vk8BRCHubBGIYyxnAZeDCLVAcGFsUrFD7NuNx077VV25ZvzhKYue9VyapiABmkjTJ%2FrvkX2CyBFVERum6r2wJ9%2Fb5Zkhi0t4SChCnbzj3Obhw4Vr7XkHzNbslh479WsjLGDOJIOOADTxpKuwWbZEigtq5am8Xyr3g7%2F6QHWf6gSWnCA%3D%3D for .zju.edu.cn/>, <Cookie JSESSIONID=0B386AA08358A38A29D90158A0FB57CB for service.zju.edu.cn/>, <Cookie language= for service.zju.edu.cn/>, <Cookie route=dabe532e66c0719d87f039027e7fbb36 for service.zju.edu.cn/>, <Cookie route=53c6d4f901872907b857cf510a994371 for zdbk.zju.edu.cn/>, <Cookie JSESSIONID=7CB7044D8DD016434CFBB6C4A8E860CA for zdbk.zju.edu.cn/jwglxt>, <Cookie JSESSIONID=05B5A0E4440B4AD811BF202AF911CA15.cas16 for zjuam.zju.edu.cn/cas>]>


{'Server': 'Tengine/2.1.2', 'Date': 'Mon, 23 Jun 2025 06:23:05 GMT', 'Content-Type': 'text/html;charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'Set-Cookie': 'JSESSIONPREJSDM=h%40Q%2CUB0Z%3AO%5EdJj%22%7BHZ9X0R0B0EF3Z%3AO%5E; Path=/jwglxt/xtgl; HttpOnly', 'Content-Language': 'zh-CN', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Credentials': 'true', 'Content-Encoding': 'gzip'}

In [25]:
print(res.cookies)

<RequestsCookieJar[]>


In [ ]:
gnmkdm = 'N508301'

headers = {
    'User-Agent': 'Mozilla/5.0 (Linux; Android 10; Redmi K30 Pro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Mobile Safari/537.36',
}

res = session.post(url=f'https://zdbk.zju.edu.cn/jwglxt/cxdy/xscjcx_cxXscjIndex.html?doType=query&gnmkdm={gnmkdm}&su={username}', data={
    'xn': None,
    'xq': None,
    'zscjl': None,
    'zscjr': None,
    '_search': 'false',
    'nd': str(int(time.time() * 1000)),
    'queryModel.showCount': 15,
    'queryModel.currentPage': 1,
    'queryModel.sortName': 'xkkh',
    'queryModel.sortOrder': 'asc',
    'time': 1,
}, headers=headers)

new_score = res.json()['items']